In [1]:
from skbio.alignment import local_pairwise_align_ssw
from skbio import DNA
import re

def binary_alignment_search(query, target, query_i=0, current_points=[]):
    if len(query)<8:        
        return current_points
    
    breaking_points = []
    alignment = local_pairwise_align_ssw(DNA(query),DNA(target))
    string = str(alignment[0][0])
    breaking_points.append(query_i+alignment[2][0][0])
    
    #inner breaking points
    found = [(m.start(0), m.end(0), m.end(0)-m.start(0)) for m in re.finditer('-+', string)]
    indices = []
    for i in range(len(found)):
        if i==0:
            indices.append(query_i+alignment[2][0][0]+found[0][0])
        else:
            indices.append(query_i+alignment[2][0][0]+indices[i-1]+found[i][0]-(found[i-1][0]+found[i-1][2]))
    if indices:
        breaking_points.extend(indices)
        
    #removing aligned parts from original sequence    
    query_side = query.split(str(alignment[0][0]).replace('-',''))
    target_side = target.split(str(alignment[0][1]).replace('-',''))

    if query_side[0]:
        query_i = query_i
        breaking_points.extend(binary_alignment_search(query_side[0], target_side[0], 
                                                       query_i=query_i, 
                                                       current_points=breaking_points))
    if query_side[1]:
        query_i = query_i + alignment[2][0][1] + 1
        breaking_points.extend(binary_alignment_search(query_side[1], target_side[1], 
                                                        query_i=query_i, 
                                                        current_points=breaking_points))
    return sorted(breaking_points)

In [2]:
cds = 'ATGTGCGACGAAGACGAGACCACCGCCCTCGTGTGCGACAATGGCTCCGGCCTGGTGAAAGCCGGCTTCGCCGGGGATGACGCCCCTAGGGCCGTGTTCCCGTCCATCGTGGGCCGCCCCCGACACCAGGGCGTCATGGTCGGTATGGGTCAGAAAGATTCCTACGTGGGCGACGAGGCTCAGAGCAAGAGAGGTATCCTGACCCTGAAGTACCCTATCGAGCACGGCATCATCACCAACTGGGATGACATGGAGAAGATCTGGCACCACACCTTCTACAACGAGCTTCGCGTGGCTCCCGAGGAGCACCCCACCCTGCTCACCGAGGCCCCCCTCAATCCCAAGGCCAACCGCGAGAAGATGACCCAGATCATGTTTGAGACCTTCAACGTGCCCGCCATGTACGTGGCCATCCAGGCCGTGCTGTCCCTCTACGCCTCCGGCAGGACCACCGGCATCGTGCTGGACTCCGGCGACGGCGTCACCCACAACGTGCCCATTTATGAGGGCTACGCGCTGCCGCACGCCATCATGCGCCTGGACCTGGCGGGCCGCGATCTCACCGACTACCTGATGAAGATCCTCACTGAGCGTGGCTACTCCTTCGTGACCACAGCTGAGCGCGAGATCGTGCGCGACATCAAGGAGAAGCTGTGCTACGTGGCCCTGGACTTCGAGAACGAGATGGCGACGGCCGCCTCCTCCTCCTCCCTGGAAAAGAGCTACGAGCTGCCAGACGGGCAGGTCATCACCATCGGCAACGAGCGCTTCCGCTGCCCGGAGACGCTCTTCCAGCCCTCCTTCATCGGTATGGAGTCGGCGGGCATTCACGAGACCACCTACAACAGCATCATGAAGTGTGACATCGACATCAGGAAGGACCTGTATGCCAACAACGTCATGTCGGGGGGCACCACGATGTACCCTGGGATCGCTGACCGCATGCAGAAAGAGATCACCGCGCTGGCACCCAGCACCATGAAGATCAAGATCATCGCCCCGCCGGAGCGCAAATACTCGGTGTGGATCGGCGGCTCCATCCTGGCCTCGCTGTCCACCTTCCAGCAGATGTGGATCACCAAGCAGGAGTACGACGAGGCCGGCCCTTCCATCGTCCACCGCAAATGCTTCTAG'
gene = 'GGCCACCGCAGCGGACAGCGCCAAGTGAAGCCTCGCTTCCCCTCCGCGGCGACCAGGGCCCGAGCCGAGAGTAGCAGTTGTAGCTACCCGCCCAGGTAGGGCAGGAGTTGGGAGGGGACAGGGGGACAGGGCACTACCGAGGGGAACCTGAAGGACTCCGGGGCAGAACCCAGTCGGTTCACCTGGTCAGCCCCAGGCCTGCGCCCTGAGCGCTGTGCCTCGTCTCCGGAGCCACACGCGCTTTAAAAAGGAGGCAAGACAGTCAGCCTCTGGAAATTAGACTTCTCCAAATTTTTCTCTAGCCCCTTTGGGCTCCTTTACCTGGCATGTAGGATGTGCCTAGGGAGATAAACGGTTTTGCTTTAGTTGTCGCCAAGGCAGTTCCCTTCCAAACTAGCGCTAGAGCGAATGAGCGAGCAGCCAGGACCACCCATTCTGGGTTTCCAACAGGCGAAAAGGCCCTTTCTGAGTTTGAAATGTCACAGGGTTCCTAACAGGCCACTCTTCCCTGGATGGGGTGCCAACGCCTTTCCCATGGGCATCTCCTTCCACCCTCACGCTGGCCCAGCAAGCAGGCAGTGCTGAGGCCTTATCTCCCTAGGTGACAGATGTGGTCAGGGAGGCGCAGAGAGGATGGGCACTAGCGTCCAGCTCCTGGAACAGGTGTCAGGCAGGGAGGGCAGACAGGTCTTGGGGAACATGTTCCCCTGGCTATGTGGACAGAGGACTTCTCAGTGGGGTCTCGCGACCCTGTGCCCCTTTTCCTGGTTCAGGGCAGCCCTTAGCCGGGGCAAAGGTCGAGAAGAGAACCCCTGGTCGCCGCCCTGGCAGAATTTGAGTGGCTCCGGCAGGAGATGTCCCTAGGTTCCTGGGGAGGGAGGACGTCGGGGCCAGCCAGGCTTACCCCCCCCTGCCGCTGAGACTTCTGCGCTGATGCACCGCGCCTCTTCGCGGTCTCCCTGTCCTTGCAGAAACTAGACACAATGTGCGACGAAGACGAGACCACCGCCCTCGTGTGCGACAATGGCTCCGGCCTGGTGAAAGCCGGCTTCGCCGGGGATGACGCCCCTAGGGCCGTGTTCCCGTCCATCGTGGGCCGCCCCCGACACCAGGTCAGGCTGCCCCTCCGCAGAGGGAGCCGGCTCGGGGTCCCCGCGTAAGCCAGCCTGGTGCCACCCGGAGCGGCGTTAACGGGTGCGTGGTGTCTCGGCTCTGCAGGGCGTCATGGTCGGTATGGGTCAGAAAGATTCCTACGTGGGCGACGAGGCTCAGAGCAAGAGAGGTATCCTGACCCTGAAGTACCCTATCGAGCACGGCATCATCACCAACTGGGATGACATGGAGAAGATCTGGCACCACACCTTCTACAACGAGCTTCGCGTGGCTCCCGAGGAGCACCCCACCCTGCTCACCGAGGCCCCCCTCAATCCCAAGGCCAACCGCGAGAAGATGACCCAGATCATGTTTGAGACCTTCAACGTGCCCGCCATGTACGTGGCCATCCAGGCCGTGCTGTCCCTCTACGCCTCCGGCAGGACCACCGGTGAGTGCCCGCTGCCCCCAGTCCCCTCTCCCGCCCCCGCCCCCGCCCCCGCCCCCGCCGCTAGCGCTGAGAGCCTAGCCTCGGCCTCGCCCCCAGCCACTCACTCTCTCCCGCGCGCGCACAGGCATCGTGCTGGACTCCGGCGACGGCGTCACCCACAACGTGCCCATTTATGAGGGCTACGCGCTGCCGCACGCCATCATGCGCCTGGACCTGGCGGGCCGCGATCTCACCGACTACCTGATGAAGATCCTCACTGAGCGTGGCTACTCCTTCGTGACCACAGGTGCGCGGCGCCCCTGCACCCCGGGCGGGAGGGCCGGCGGCGGCCCCTGAGTGAGGGCTCCTCTCCTGCTTCTGCCCTCCGCAGCTGAGCGCGAGATCGTGCGCGACATCAAGGAGAAGCTGTGCTACGTGGCCCTGGACTTCGAGAACGAGATGGCGACGGCCGCCTCCTCCTCCTCCCTGGAAAAGAGCTACGAGCTGCCAGACGGGCAGGTCATCACCATCGGCAACGAGCGCTTCCGCTGCCCGGAGACGCTCTTCCAGCCCTCCTTCATCGGTGAGCCCCGCTCGCCCTCGCCCCGGCCCCCAGGCCCCGCCGGCCCCCGGCGCCCCGAGCTTCTGCTCACGCTCCCCGCCGCGGTCCCCAGGTATGGAGTCGGCGGGCATTCACGAGACCACCTACAACAGCATCATGAAGTGTGACATCGACATCAGGAAGGACCTGTATGCCAACAACGTCATGTCGGGGGGCACCACGATGTACCCTGGGATCGCTGACCGCATGCAGAAAGAGATCACCGCGCTGGCACCCAGCACCATGAAGATCAAGGTGGGTGGTGGCCTGCGCGGGCTGTCGGCGGGGTGGGCTCCAGGGTGAGGTCTCCCCACCTCACGCGCTGTCTTGCAGATCATCGCCCCGCCGGAGCGCAAATACTCGGTGTGGATCGGCGGCTCCATCCTGGCCTCGCTGTCCACCTTCCAGCAGATGTGGATCACCAAGCAGGAGTACGACGAGGCCGGCCCTTCCATCGTCCACCGCAAATGCTTCTAGACACACTCCACCTCCAGCACGCGACTTCTCAGGACGACGAATCTTCTCAATGGGGGGGCGGCTGAGCTCCAGCCACCCCGCAGTCACTTTCTTTGTAACAACTTCCGTTGCTGCCATCGTAAACTGACACAGTGTTTATAACGTGTACATACATTAACTTATTACCTCATTTTGTTATTTTTCGAAACAAAGCCCTGTGGAAGAAAATGGAAAACTTGAAGAAGCATTAAAGTCATTCTGTTAAGCTGCGTAAA'
result = binary_alignment_search(cds, gene)

In [3]:
result

[0, 126, 453, 613, 807, 988]

In [4]:
for i, j in enumerate(result):
    if j == result[-1]:
        print('/'+cds[j:]+'/')
    else:
        print('/'+cds[j:result[i+1]]+'/')

/ATGTGCGACGAAGACGAGACCACCGCCCTCGTGTGCGACAATGGCTCCGGCCTGGTGAAAGCCGGCTTCGCCGGGGATGACGCCCCTAGGGCCGTGTTCCCGTCCATCGTGGGCCGCCCCCGACAC/
/CAGGGCGTCATGGTCGGTATGGGTCAGAAAGATTCCTACGTGGGCGACGAGGCTCAGAGCAAGAGAGGTATCCTGACCCTGAAGTACCCTATCGAGCACGGCATCATCACCAACTGGGATGACATGGAGAAGATCTGGCACCACACCTTCTACAACGAGCTTCGCGTGGCTCCCGAGGAGCACCCCACCCTGCTCACCGAGGCCCCCCTCAATCCCAAGGCCAACCGCGAGAAGATGACCCAGATCATGTTTGAGACCTTCAACGTGCCCGCCATGTACGTGGCCATCCAGGCCGTGCTGTCCCTCTACGCCTCCGGCAGGACCACC/
/GGCATCGTGCTGGACTCCGGCGACGGCGTCACCCACAACGTGCCCATTTATGAGGGCTACGCGCTGCCGCACGCCATCATGCGCCTGGACCTGGCGGGCCGCGATCTCACCGACTACCTGATGAAGATCCTCACTGAGCGTGGCTACTCCTTCGTGACCA/
/CAGCTGAGCGCGAGATCGTGCGCGACATCAAGGAGAAGCTGTGCTACGTGGCCCTGGACTTCGAGAACGAGATGGCGACGGCCGCCTCCTCCTCCTCCCTGGAAAAGAGCTACGAGCTGCCAGACGGGCAGGTCATCACCATCGGCAACGAGCGCTTCCGCTGCCCGGAGACGCTCTTCCAGCCCTCCTTCATC/
/GGTATGGAGTCGGCGGGCATTCACGAGACCACCTACAACAGCATCATGAAGTGTGACATCGACATCAGGAAGGACCTGTATGCCAACAACGTCATGTCGGGGGGCACCACGATGTACCCTGGGATCGCTGACCGCATGCAGAAAGAGATCACCGCGCTGGCACCCAGCACCATGAAGATC

In [5]:
gene.index('GGTATGGAGTCGGCGGGCATTCACGAGACCACCTACAACAGCATCATGAAGTGTGACATCGACATCAGGAAGGACCTGTATGCCAACAACGTCATGTCGGGGGGCACCACGATGTACCCTGGGATCGCTGACCGCATGCAGAAAGAGATCACCGCGCTGGCACCCAGCACCATGAAGATCA')

2195

In [6]:
gene.index(cds[988:])-len(cds[807:988])

2273

In [7]:
gene.index('ATCATCGCCCCGCCGGAGCGCAAATACTCGGTGTGGATCGGCGGCTCCATCCTGGCCTCGCTGTCCACCTTCCAGCAGATGTGGATCACCAAGCAGGAGTACGACGAGGCCGGCCCTTCCATCGTCCACCGCAAATGCTTCTAGACACACTCCACCTCCAGCACGCGACTTCTCAGGACGACGAATCTTCTCAATGGGGGGGCGGCTGAGCTCCAGCCACCCCGCAGTCACTTTCTTTGTAACAACTTCCGTTGCTGCCATCGTAAACTGACACAGTGTTTATAACGTGTACATACATTAACTTATTACCTCATTTTGTTATTTTTCGAAACAAAGCCCTGTGGAAGAAAATGGAAAACTTGAAGAAGCATTAAAGTCATTCTGTTAAGCTGCGTAAA')

2456

In [8]:
2273+len(cds[988:])

2419

In [9]:
len(gene)

2854

In [10]:
#https://www.ncbi.nlm.nih.gov/nuccore/NM_001100.4
#https://rest.ensembl.org/sequence/id/ENSG00000143632?content-type=text/x-fasta;multiple_sequences=true;type=cds;format=fasta